In [1]:
import re
import sys
from pathlib import Path
from collections import Counter
from datetime import datetime, time

import numpy as np
import modin.pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import ray

ray.init()

2021-09-27 22:44:46,104	INFO services.py:1263 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '143.248.80.23',
 'raylet_ip_address': '143.248.80.23',
 'redis_address': '143.248.80.23:6379',
 'object_store_address': '/tmp/ray/session_2021-09-27_22-44-44_517432_1850814/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-09-27_22-44-44_517432_1850814/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-09-27_22-44-44_517432_1850814',
 'metrics_export_port': 64281,
 'node_id': 'a8a8212c6e8e14883dab40a352d5d4533d5b51d9cfbf1d24bc7baa3a'}

In [2]:
ROOT = Path('/home/jun0/Projects/emart24')
sys.path.append(str(ROOT))
print(sys.path)

['/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages/ray/thirdparty_files', '/home/jun0/Projects/emart24/emart24_codes/RJY', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages/ray/pickle5_files', '/home/jun0/Projects/emart24/emart24_codes/RJY', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles', '/home/jun0/.vscode-server/extensions/ms-toolsai.jupyter-2021.8.2041215044/pythonFiles/lib/python', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python38.zip', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/lib-dynload', '', '/home/jun0/.local/lib/python3.8/site-packages', '/home/jun0/.pyenv/versions/miniconda3-latest/envs/cuda111/lib/python3.8/site-packages', '/home/jun0/.local/lib/python3.8/site-packages/IPython/extensions', '/home/jun0/.ipython', '/home/jun0/Projects/e

# 분류 체계 확장

In [3]:
# 카테고리 데이터
category = pd.read_excel(ROOT / 'data/category.xlsx')
category.columns = ['KAN_CODE', '전체분류', '현재분류명', '상품예시']

# 상품 마스터 데이터
item = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
item['점포발주입수수량'] = item['점포발주입수수량'].fillna(0)
item['공급가'] = item['공급가'].fillna(0).apply(lambda x: round(float(x)))
item['매가'] = item['매가'].fillna(0)
item['점포유통기한일수'] = item['점포유통기한일수'].fillna(0)
item = item.astype({'점포발주입수수량': int, '공급가': int, '매가': int, '점포유통기한일수': int})

# shapes
category.shape, item.shape

((3062, 4), (49299, 46))

In [5]:
category.head(2)

,KAN_CODE,전체분류,현재분류명,상품예시
0,1010101,가공식품>조미료>종합조미료>천연/발효조미료,천연/발효조미료,"다시다, 미원, 멸치가루, 버섯가루"
1,1010102,가공식품>조미료>종합조미료>식초,식초,"과일초, 발효초, 감식초, 사과식초, 현미식초, 홍초"


In [68]:
contained = item.fillna('').apply(lambda row: (row['GDS_SLPNT_DSCRT'] in row['GDS_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
contained_reverse = item.fillna('').apply(lambda row: (row['GDS_DSCRT'] in row['GDS_SLPNT_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)
only_dscrt = item['GDS_DSCRT'].isna() & ~item['GDS_SLPNT_DSCRT'].isna()
only_slpnt = ~item['GDS_DSCRT'].isna() & item['GDS_SLPNT_DSCRT'].isna()

In [88]:
item[item.fillna('').apply(lambda row: (row['GDS_SLPNT_DSCRT'] != row['GDS_DSCRT']) & (row['GDS_DSCRT'] in row['GDS_SLPNT_DSCRT']) & bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']]

,상품명,GDS_DSCRT,GDS_SLPNT_DSCRT
55,에너자이저)맥스AAA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
123,에너자이저)맥스AA2입,에너자이저 국내 판매 1위 건전지,[사이드진열 Best 추천 상품] 에너자이저 국내 판매 1위 건전지
704,천지개벽100ml,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...,- 트렌드 : 숙취해소음료 카테고리 내 구색 다양화를 통한 고객 니즈 확보가 필요함...
753,하겐다즈)화이트블레스케익,"- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛...","- 향긋한 그린티 아이스크림과 부드러운 바닐라 아이스크림, 달콤한 위핑크림과 초코렛..."
9174,마즈)스니커즈땅콩헤이즐넛,"고소함으로 가득찬 헤이즐넛, 두개로 나누어 먹을 수 있는 초코바","1. 고소함으로 가득찬 헤이즐넛, 두개로 나누어 먹을 수 있는 초코바 2. 학원가..."
...,...,...,...
48592,스티커)컬러풀하트스티커,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 다양한 모양의 하트 스티...,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 다양한 모양의 하트 스티...
48593,스티커)마스크데코스티커(과일),1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 밋밋한 마스크를 개성있게...,1. 스티커 제품은 회전율과 매출이 높은 상품입니다. 2. 밋밋한 마스크를 개성있게...
48594,바른생각)콘돔익스트림에어핏기획(1P),1.바른생각)인기콘돔4종클립스트립(기획상품) 구성품 / 상품 발주 불가,1.바른생각)인기콘돔4종클립스트립(기획상품) 구성품 / 상품 발주 불가 4. 진열...
49086,신스무디킹)글래드랩(포장용),1. 스무디킹 운영용 소모품 재료,1. 스무디킹 운영용 소모품 재료


In [84]:
item[item['GDS_DSCRT'].isna() & item['GDS_SLPNT_DSCRT'].isna()].shape

(19670, 48)

In [81]:
item[item.fillna('').apply(lambda row: bool(row['GDS_DSCRT']) & bool(row['GDS_SLPNT_DSCRT']), axis=1)].shape

(13869, 48)

In [83]:
3624 / 13869

0.2613021847285313

In [69]:
item[contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(3578, 3)

In [70]:
item[only_dscrt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(8, 3)

In [71]:
item[only_slpnt][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(15752, 3)

In [72]:
item[contained_reverse][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(3624, 3)

In [73]:
item[~contained][['상품명', 'GDS_DSCRT', 'GDS_SLPNT_DSCRT']].shape

(45721, 3)

In [9]:
temp = pd.read_csv(ROOT / 'data/prep/02.설명이추가된 상품마스터_20210325.csv', sep='|', dtype=str)
temp[temp['점포발주입수수량'].isna() | temp['공급가'].isna() | temp['매가'].isna() | temp['점포유통기한일수'].isna()][['점포발주입수수량', '공급가', '매가', '점포유통기한일수']]

,점포발주입수수량,공급가,매가,점포유통기한일수
27089,NaN,333,450,NaN


In [41]:
# 전체 카테고리
all_category = []
temp = pd.DataFrame(category['전체분류'].apply(lambda row: row.split('>')).tolist())
for col in temp.columns:
    all_category += temp[col].unique().tolist()
unique_category = set(all_category)
len(all_category), len(unique_category)

(3576, 3427)

In [4]:
# 전체분류 마지막 항목과 현재분류명 일치하지 않는 케이스
category[~(category['전체분류'].apply(lambda row: row.split('>')[-1]) == category['현재분류명'])]

,KAN_CODE,전체분류,현재분류명,상품예시


In [35]:
# 전처리해야할 대상들
category[category['전체분류'].duplicated(keep=False)]

,KAN_CODE,전체분류,현재분류명,상품예시


In [42]:
# category의 아이템
category_item = [x for lst in category['상품예시'].apply(lambda row: [x.strip() for x in row.split(',')]).tolist() for x in lst]
# '' 제거
category_item = list(filter(lambda x: x != '', category_item))
category_item_unique = set(category_item)
category_item_unique = list(category_item_unique)[1:]
len(category_item), len(category_item_unique)

(9353, 9140)

In [45]:
# unique 한 상품명
item_unique_bool = ~item['상품명'].duplicated()
# 분류가 겹치는 아이템
overlap = item_unique_bool & (item['대분류명'].isin(unique_category) | item['중분류명'].isin(unique_category) | item['소분류명'].isin(unique_category))
# 단순 일치하는 상품명
exact_match_bool = item_unique_bool & item['상품명'].isin(category_item_unique)
# 상품명에 포함된 상품예시
contain_bool = item_unique_bool & item['상품명'].apply(lambda x: bool([i for i in category_item_unique if i in x]))
item['상품예시'] = item['상품명'].apply(lambda x: [i for i in category_item_unique if i in x])
item['카테고리'] = ''

item[overlap].shape, item[exact_match_bool].shape, item[contain_bool].shape, item[overlap | contain_bool].shape

((17830, 47), (283, 47), (23738, 47), (31591, 47))

In [66]:
item[item['상품예시'].apply(lambda row: bool([x for x in row if len(x) > 1]))][['PLU_CD', '상품명', '상품예시', '카테고리']].sample(100)

,PLU_CD,상품명,상품예시,카테고리
36017,8801062896943,롯데)스키피땅콩스트로젤,[땅콩],
10781,8806075265885,자주)물병에코(오렌지),"[물병, 오렌지]",
16570,8809073333556,완구)호키동물피아노,[피아노],
40371,8801198180282,영실업)마이크로휠(싱글팩),"[마이크, 마, 실]",
17101,8803284608199,피코크)리치핫초코10입,[리치],
...,...,...,...,...
11565,8801047163053,명품)한우야채죽,"[야채죽, 한우]",
29534,8801047154761,동원)돌후룻샐러드파우치115g,[샐러드],
42186,5702016618259,레고)21158 판다 가족의 놀이터,[레고],
32226,1500000012847,바르다)참치&크림치즈호두김밥,"[호두, 크림치즈]",


In [92]:
temp_item_list = ['징', '오일', '클렌징오일', '참기름', '수세미', '시가', '주방가위', '가위']
temp_bool = category['상품예시'].apply(lambda row: any([x.strip() in temp_item_list for x in row.split(',')]))
category[temp_bool].head(10)

,KAN_CODE,전체분류,현재분류명,상품예시
36,1010501,가공식품>조미료>유지류>참기름,참기름,"참기름, 생참기름, 맛참기름"
755,3110106,일상용품>주방용품>조리기구>주방가위,주방가위,"주방가위, 식가위, 식사가위"
793,3110405,일상용품>주방용품>주방소모품>수세미,수세미,"수세미, 철수세미, 천수세미"
943,3180499,일상용품>공구>절삭공구>기타절삭공구,기타절삭공구,"가위, 트리머, 다이스, 대패, 루터, 샌더기, 톱, 조각기, 직소기, 커터, 탭"
1040,3200305,일상용품>화장품>스킨케어화장품>오일,오일,"오일, 스킨오일, 페이셜오일"
1178,3201407,일상용품>화장품>클렌징>클렌징오일,클렌징오일,"클렌징오일, 오일형클렌져, 클렌져오일"
1197,3220103,일상용품>담배용품>담배>시가,시가,"시가, 바닐라시가, 플레이버시가"
1495,6010207,교육/문화용품>문구/사무용품>일반사무용품>가위,가위,"가위, 문서가위, 사무용가위"
1769,6040499,교육/문화용품>악기>국악악기>기타국악악기,기타국악악기,"대취타, 소고, 징"
1783,6040704,교육/문화용품>악기>타악기>징,징,"징, 국악용징, 학생용징"


In [ ]:
# 상품 설명 보기
item[['GDS_DSCRT', 'GDS_SLPNT_DSCRT']].drop_duplicates().values.tolist()

# 거래 데이터

In [3]:
# 거래데이터
# emart24 = ROOT / 'data/emart24Data'
# f = emart24 / '001_TRAN/IFAI_001_TRAN_00115.txt'
# f = emart24 / '007_DAYQTY/IFAI_007_DAYQTY_00115.txt'
# f = emart24 / '008_STRPUR/IFAI_008_STRPUR_00115.txt'
# f = emart24 / '009_STRORD/IFAI_009_STRORD_00115.txt'
# f = emart24 / 'IFAI_100_RCMQTY_03193.txt'
files = (ROOT / 'data/emart24Data/001_TRAN').glob('*.txt')
transaction = pd.DataFrame()
for i, file in enumerate(files):
    transaction = pd.concat([transaction, pd.read_csv(file, sep='|', low_memory=False)], 0)

# 파일 수, 데이터 건수
i + 1, transaction.shape

(226, (80294983, 95))

In [4]:
transaction.iloc[:, :10]

,TRAN_YMD,COM_CD,STORE_CD,POS_NO,TRAN_NO,SYS_YMDHMS,TRAN_TYPE,TRAN_TYPE_NM,TRAN_KIND,TRAN_KIND_NM
0,20190101,1002,4890,1,1665,20190101000303,0,통상,0,판매
1,20190101,1002,4890,1,1666,20190101000352,0,통상,0,판매
2,20190101,1002,4890,1,1668,20190101000810,0,통상,0,판매
3,20190101,1002,4890,1,1670,20190101001001,0,통상,0,판매
4,20190101,1002,4890,1,1670,20190101001001,0,통상,0,판매
...,...,...,...,...,...,...,...,...,...,...
303436,20210131,1002,3398,1,1292,20210131234740,0,통상,0,판매
303437,20210131,1002,3398,1,1292,20210131234740,0,통상,0,판매
303438,20210131,1002,3398,1,1293,20210131234838,0,통상,0,판매
303439,20210131,1002,3398,1,1294,20210131235229,0,통상,0,판매


In [5]:
# 상품 분류 데이터
item1 = pd.read_excel(ROOT / 'data/prep/분류명겹침_매핑.xlsx')
item2 = pd.read_csv(ROOT / 'data/prep/item_category_matched_with_example.csv')
item1.shape, item2.shape

((9088, 48), (29265, 49))

In [6]:
items = pd.concat([item1, item2], 0)[['PLU_CD', '상품명', '전체분류', '현재분류명', '상품예시']]
items['상품명'].unique().shape, items['PLU_CD'].unique().shape, items.shape

((23120,), (25248,), (38353, 5))

In [7]:
items.sample(5)

,PLU_CD,상품명,전체분류,현재분류명,상품예시
8266,8809650793643,대정)리코타크렌베리샌드2편,가공식품>빵류/떡류>빵류>조리빵,조리빵,NaN
1284,8801033793394,정식품)입안가득바나나200ml,가공식품>음료류>기능성/건강음료>두유,두유,NaN
2256,8801117773700,오리온)뉴팝스위트콘1200,가공식품>과자류>비스킷>스낵,스낵,NaN
17379,8802039210526,면사랑)닭가슴살치즈크림스파게티2인분,신선식품>축산물>조류>닭고기(부위별),닭고기(부위별),닭가슴살
4274,8801097150010,포카리스웨트캔245ml,일상용품>애완용품>강아지 간식>음료,음료,NaN


In [ ]:
items.sort_values('PLU_CD').to_csv(ROOT / 'data/prep/items.csv', index=False)

In [18]:
transaction['ITEM_NM'].unique().shape, transaction['PLU_CD'].unique().shape

((24916,), (17860,))

In [8]:
trans_filtered = transaction[transaction['PLU_CD'].isin(items['PLU_CD'].unique())]
trans_filtered.shape

(49099013, 95)

In [8]:
trans_filtered['PLU_CD'].unique().shape, trans_filtered['ITEM_NM'].unique().shape, 

In [39]:
trans_filtered.iloc[:10, :].merge(items[~items['PLU_CD'].duplicated()], how='left', on='PLU_CD')[['PLU_CD', 'ITEM_NM', '상품명', '전체분류', '현재분류명', '상품예시']]

,PLU_CD,ITEM_NM,상품명,전체분류,현재분류명,상품예시
0,8801094202606,스프라이트500ml,스프라이트500ml,일상용품>애완용품>강아지 간식>음료,음료,NaN
1,6901035603232,비어케이)칭따오캔맥주500ML,비어케이)칭따오캔맥주500ML,가공식품>주류>양조주>맥주,맥주,NaN
2,4901004006813,롯데아사히)캔맥주500ML,롯데아사히)캔맥주500ML,가공식품>주류>양조주>맥주,맥주,NaN
3,8809429238283,SF)뉴전주비빔삼각김밥2편,SF)뉴전주비빔삼각김밥2편,가공식품>즉석/편의식품>편의식품>김밥,김밥,삼각김밥
4,7038516550309,조르단)인디비주얼칫솔,조르단)인디비주얼칫솔,일상용품>위생용품>구강위생용품>칫솔,칫솔,NaN
5,8801056018900,칸타타)아메리카노캔275ml,칸타타)아메리카노캔275ml,일상용품>애완용품>강아지 간식>음료,음료,NaN
6,8801048311002,하이트)이슬톡톡복숭아355ML,하이트)이슬톡톡복숭아355ML,가공식품>주류>양조주>맥주,맥주,NaN
7,6901035603232,비어케이)칭따오캔맥주500ML,비어케이)칭따오캔맥주500ML,가공식품>주류>양조주>맥주,맥주,NaN
8,8801056080457,칸타타)아메리카노캔390ml,칸타타)아메리카노캔390ml,일상용품>애완용품>강아지 간식>음료,음료,NaN
9,8808244201045,광동)삼다수2L,광동)삼다수2L,일상용품>애완용품>강아지 간식>음료,음료,NaN


In [11]:
trans_filtered['YMD'] = trans_filtered['SYS_YMDHMS'].apply(lambda row: datetime.strptime(f'{row // 1000000}', '%Y%m%d'))
trans_filtered['time'] = trans_filtered['SYS_YMDHMS'].apply(lambda row: time(hour=row % 1000000 // 10000).strftime('%H:%M:%S'))

In [40]:
trans_filtered = trans_filtered.merge(items[~items['PLU_CD'].duplicated()], how='left', on='PLU_CD')

In [45]:
trans_filtered.shape

(128091299, 101)

In [42]:
for g, df in trans_filtered.groupby('YMD'):
    # print(g, df.shape)
    g = g[:10].replace('-', '')
    df.to_csv(ROOT / f'data/prep/transactions/{g}.csv', index=False)
    # df.to_parquet('output.parquet', )

In [57]:
# 파일명 잘못된거 바꾸기
# for f in sorted((ROOT / 'data/prep/transactions').glob('*.csv')):
#     new_f = f.parent / f"{f.name[:10].replace('-', '')}.csv"
#     pd.read_csv(f).to_csv(new_f, index=False)

In [59]:
trans_filtered = pd.DataFrame()
for f in sorted((ROOT / 'data/prep/transactions').glob('*.csv')):
    # print(g, df.shape)
    trans_filtered = pd.concat([trans_filtered, pd.read_csv(f)])
trans_filtered.shape

(49099013, 101)

In [68]:
temp_df = trans_filtered['YMD'].value_counts().reset_index()
temp_df['x'] = temp_df['index'].apply(lambda row: datetime.strptime(str(row), '%Y-%m-%d'))
fig = px.bar(temp_df, x='x', y='YMD')
fig.show()

In [ ]:
temp_df = trans_filtered['time'].value_counts().reset_index().sort_values('index')
# temp_df['index'] = temp_df['index'].apply(lambda row: time(hour=int(row[:2]), minute=int(row[2:4]), second=int(row[4:])).strftime('%H:%M:%S'))
temp_df['x'] = temp_df['index'].apply(lambda row: time(hour=row).strftime('%H:%M:%S'))
fig = px.bar(temp_df, x='index', y='time')
fig.show()